# 一、项目介绍  
1、项目背景：本次分析数据覆盖 11 月 25 日至 12 月 3 日，包含周末及 “双十二” 预热期，此阶段用户行为、商品交互等数据可能呈现特殊时段特征。为支撑平台运营决策、提升业务效能，需通过数据下钻分析，探究该时段用户行为规律与品类运营表现。  
2、项目目标：  
* 挖掘用户流量、用户行为、用户画像以及商品特征。  
* 建立指标体系或者业务模型，并实现数据可视化。  
* 揪出业务问题并给出一定的优化建议。  

3、数据介绍：该数据为淘宝阿里巴巴提供的一个淘宝用户行为数据集，发布在阿里巴巴天池https://tianchi.aliyun.com/dataset/649  
关于数据每一列的内容主要如下：  

![Image Name](https://cdn.kesci.com/upload/t02m0p9kww.png?imageView2/0/w/1280/h/1280)  


# 二、技术栈  
1、数据清洗和主表搭建：Python（pandas）  
2、数据处理和建表查询：Mysql  
3、数据分析可视化：Tableau

# 三、数据清洗与主表搭建  
由于数据量过于庞大，达到惊人的一亿行，用Exce处理直接卡死，SQL增删改又存在代码冗杂，效率低下的问题，所以相比之下用Python的pandas库处理较为合理。  
由于硬件问题，无法分析太多数据，所以汲取其他博主的经验，只采用源数据的前300w行，但是如果将来需要扩展数据，用本文的代码重新跑一遍就好了。

In [ ]:
import pandas as pd
import time

start_time = time.time()

input_file = r'E:/淘宝电商用户行为/tbub_all.csv'
output_file = r'E:/淘宝电商用户行为/tbub_.csv'

n_rows = 3000000

df = pd.read_csv(input_file, nrows=n_rows)

df.to_csv(output_file, index=False)

elapsed_time = time.time() - start_time
print(f"数据处理完成！共耗时 {elapsed_time:.2f} 秒。")
print(f"已成功保存 {len(df):,} 行数据到新文件。")

保存好数据后，开始正式清洗数据：增加列标题；转换日期格式，增加日期、小时、星期列；限定日期范围为2017-11-25到2017-12-03；检查缺失值，初步检查重复值。  
确认没有缺失值，过滤了1条重复值。

In [ ]:
import pandas as pd

# 定义列标题（避开SQL关键字，更适合数据库操作）
column_names = [
    'user_id',          # 用户ID
    'product_id',       # 商品ID
    'category_id',      # 商品类目ID
    'action_type',      # 行为类型
    'event_timestamp'   # 事件时间戳
]

# 读取CSV文件并指定列名
df = pd.read_csv(
    r"E:\淘宝电商用户行为\tbub_.csv",
    names=column_names
)

# 将时间戳转换为日期时间格式（UTC+8时区）
df['times'] = pd.to_datetime(df['event_timestamp'], unit='s') + pd.Timedelta(hours=8)

# 提取日期和时间信息（避开SQL关键字）
df['dates'] = df['times'].dt.date       # 事件日期
df['hours'] = df['times'].dt.hour       # 事件小时
df['weekdays'] = df['times'].dt.day_name()  # 事件星期

print("数据基本信息：")
df.info()

# 查看数据集行数和列数
rows, columns = df.shape

if rows > 0:
    # 查看数据集行数和列数
    print(f"\n数据包含 {rows} 条记录，{columns} 个字段")

    # 查看数据集前几行（避开SQL关键字的列名）
    print("\n数据前几行内容信息：")
    print(df[['user_id', 'product_id', 'action_type', 'times', 'dates', 'hours', 'weekdays']].head().to_csv(sep='\t', na_rep='nan'))


# 定义日期范围
start_date = pd.to_datetime('2017-11-25 00:00:00')
end_date = pd.to_datetime('2017-12-03 23:59:59')

# 过滤日期范围内的数据
original_rows = len(df)
df = df[(df['times'] >= start_date) & (df['times'] <= end_date)]
filtered_rows = len(df)
filtered_percentage = (filtered_rows / original_rows) * 100 if original_rows > 0 else 0

print("\n过滤后数据基本信息：")
df.info()

# 检查缺失值
print("\n缺失值统计：")
print(df.isnull().sum())

# 检查重复值并删除
print("\n重复值统计：")
duplicate_count = df.duplicated().sum()
print(f"删除前重复值数量：{duplicate_count}")
df = df.drop_duplicates()
print(f"删除后重复值数量：{df.duplicated().sum()}")

# 保存处理后的数据（避开SQL关键字的列名）
df.to_csv(r"E:\淘宝电商用户行为\tbub.csv", index=False)

# 四、数据处理和建表查询  
## 4.1数据导入  
在数据库管理工具中导入清洗后的数据 （Navicat）  

打开Navicat，新建查询，选择Mysql，填写自定义连接名称和下载Mysql时候设置的密码。  

![Image Name](https://cdn.kesci.com/upload/t02njz5e3e.png?imageView2/0/w/1280/h/1280)  


打开连接，右键新建数据库，自定义数据库名称，字符集选择utf8mb4，这是一种字符编码方式，是UTF-8编码的超集，主要用于更全面地表示文本字符；排序规则我选择utf8mb4_0900_ai_ci，这是 MySQL 8.0 引入的一种字符排序规则，不区分大小写且不区分重音。  

![Image Name](https://cdn.kesci.com/upload/t03ru95t6u.png?imageView2/0/w/960/h/960)  

下拉数据库，右键表，选择【导入向导】，点击后导入刚才Python清洗好的tbub.csv文件，  
一直点击【下一步】，最后点击【开始】按钮进行导入。  

![Image Name](https://cdn.kesci.com/upload/t03rvitgjt.png?imageView2/0/w/960/h/960)  

导入完成后，概览表tbub，没有问题后开始修改字段类型：

In [ ]:
ALTER TABLE  tbub
MODIFY COLUMN dates DATE;

In [ ]:
ALTER TABLE your_table_name 
MODIFY COLUMN full_datetime DATETIME;

上次python是全行完全相同的去重，现在再根据用户名，商品id，时间去重

In [ ]:
SELECT user_id,product_id,times,dates
FROM tbub
group by  user_id,product_id,times,dates
having count(*)>1

确认重复值为0，  
现在，就可以正式开始数据探索了！

## 4.2用户分析  
### 4.2.1用户流量  
分析页面浏览量PV和独立访客数UV，  
创建指标PV/UV计算平均每个独立访客的页面浏览次数，这里称为浏览深度。  


In [ ]:
create table df_pv_uv 
(
  dates varchar(10),
  PV int(9),
  UV int(9),
  PVUV decimal(10,2)
);

INSERT into df_pv_uv 
select 
dates,
count(if(action_type='pv',1,null)) as PV,
count(distinct user_id) as UV, 
round(count(if(action_type='pv',1,null))/count(distinct user_id),2) as 'PVUV'
from tbub 
group by dates;

### 4.2.2 用户留存  
计算用户单日和三日的留存率  
次日留存率：  


In [ ]:
CREATE TABLE df_retention_1 (dates CHAR(10), retention_1 FLOAT);
INSERT INTO df_retention_1 SELECT
  ub1.dates,
  count(ub2.user_id) / count(ub1.user_id) AS retention_1
FROM
  (SELECT DISTINCT user_id, dates FROM tbub) AS ub1
  LEFT JOIN (SELECT DISTINCT user_id, dates FROM tbub) AS ub2 ON ub1.user_id = ub2.user_id
  AND ub2.dates = DATE_ADD(ub1.dates, INTERVAL 1 DAY)
GROUP BY
  ub1.dates

三日留存率：

In [ ]:
create table df_retention_3
(
  dates char(10),
  retention_3 FLOAT
  );
  
INSERT into df_retention_3
select ub1.dates,count(ub2.user_id)/count(ub1.user_id) as retention_3
FROM
  (SELECT DISTINCT user_id, dates FROM tbub) AS ub1
left join 
(select DISTINCT user_id,dates 
from tbub) as ub2 
on ub1.user_id = ub2.user_id 
and ub2.dates = DATE_ADD(ub1.dates,INTERVAL 3 day)
group by ub1.dates 

### 4.2.3用户行为  
以时间序列的逻辑，汇总每一天每个小时执行PV、CART、FAV、BUY的次数:  



In [ ]:
CREATE TABLE df_timeseries (
  dates VARCHAR (10),
  hours INT (9),
  PV INT (9),
  CHRT INT (9),
  FAV INT (9),
  BUY INT (9)
);
INSERT INTO df_timeseries SELECT
  dates,
  hours,
  count(IF(action_type = 'pv', 1, NULL)) AS PV,
  count(IF(action_type = 'cart', 1, NULL)) AS CART,
  count(IF(action_type = 'fav', 1, NULL)) AS FAV,
  count(IF(action_type = 'buy', 1, NULL)) AS BUY
FROM
  tbub
GROUP BY
  dates,
  hours
ORDER BY
  dates ASC,
  hours ASC

### 4.2.4 用户路径  
以单个用户的逻辑，按照用户id和品类id分组，统计每一个用户是否有执行PV、CART、FAV、BUY这四个步骤，再组合这些步骤成为用户的行为路径。  
会存在用户没有浏览行为直接购买的情况（如‘/-/-/-购买'），这是因为数据是从2017-11-25日开始，用户在2017-11-25日之前已经进行了其他步骤，我们不得而知，所以将这种情况需要排除，我们仅研究存在浏览行为的用户路径。  


In [ ]:
CREATE TABLE path AS WITH ubt AS (
  SELECT
    user_id,
    category_id,
    COUNT(IF(action_type = 'pv', 1, NULL)) AS PV,
    COUNT(IF(action_type = 'fav', 1, NULL)) AS FAV,
    COUNT(IF(action_type = 'cart', 1, NULL)) AS CART,
    COUNT(IF(action_type = 'buy', 1, NULL)) AS BUY
  FROM
    tbub
  GROUP BY
    user_id,
    category_id
),
ifubt AS (
  SELECT
    user_id,
    category_id,
    IF(PV > 0, 1, 0) AS ifpv,
    IF(FAV > 0, 1, 0) AS iffav,
    IF(CART > 0, 1, 0) AS ifcart,
    IF(BUY > 0, 1, 0) AS ifbuy
  FROM
    ubt
  GROUP BY
    user_id,
    category_id
),
user_path AS (SELECT user_id, category_id, CONCAT(ifpv, iffav, ifcart, ifbuy) AS path FROM ifubt) SELECT
  user_id,
  category_id,
  path,
  CASE
    WHEN path = 1101 THEN
      '浏览-收藏-/-购买'
    WHEN path = 1011 THEN
      '浏览-/-加购-购买'
    WHEN path = 1111 THEN
      '浏览-收藏-加购-购买'
    WHEN path = 1001 THEN
      '浏览-/-/-购买'
    WHEN path = 1010 THEN
      '浏览-/-加购-/'
    WHEN path = 1100 THEN
      '浏览-收藏-/-/'
    WHEN path = 1110 THEN
      '浏览-收藏-加购-/'
    ELSE
      '浏览-/-/-/'
  END AS buy_path
FROM
  user_path
WHERE
  path REGEXP '^1' -- 仅研究存在浏览行为的用户路径

### 4.2.5 漏斗模型  
分别计算平台每天的各个用户行为的数量，根据路径PV-CART/FAV-BUY构建漏斗模型 

In [ ]:
create table funnel as 
SELECT 
  dates,
  COUNT(DISTINCT CASE WHEN action_type = 'pv' THEN user_id END) AS pv_num,
  COUNT(DISTINCT CASE WHEN action_type = 'cart' THEN user_id END) + 
  COUNT(DISTINCT CASE WHEN action_type = 'fav' THEN user_id END) AS cart_fav_num,
  COUNT(DISTINCT CASE WHEN action_type = 'buy' THEN user_id END) AS buy_num
FROM tbub
GROUP BY dates;

### 4.2.6 用户画像  
RMF 模型是客户关系管理（CRM）和营销分析中常用的一种客户分层模型，通过三个核心指标对客户价值进行量化评估：最近一次消费时间（Recency）：、消费频率（Frequency）、消费金额（Monetary），可以帮助企业识别高价值客户、制定精准营销策略。  
但是我们的数据集没有消费金额金额的记录，所以考虑将 “消费金额（Monetary）” 替换为 “收藏+加购（Cart&Favorite）” 后的次数，命名为RFC 模型。  
根据RFC模型将用户划分为8个类别。

In [ ]:
CREATE TABLE df_rfc AS
WITH 
r AS (
    SELECT 
        user_id,
        MAX(dates) AS recency  
    FROM tbub 
    WHERE action_type = 'buy'
    GROUP BY user_id
),
f AS (
    SELECT 
        user_id,
        COUNT(*) AS frequency  
    FROM tbub 
    WHERE action_type = 'buy'
    GROUP BY user_id
),
c AS (
    SELECT 
        user_id,
        COUNT(*) AS cart_fav_count  
    FROM tbub 
    WHERE action_type IN ('cart', 'fav')  -- 统计加购和收藏行为
    GROUP BY user_id
),
rfc_base AS (
    SELECT 
        r.user_id,
        r.recency,
        f.frequency,
        c.cart_fav_count
    FROM r
    LEFT JOIN f ON r.user_id = f.user_id
    LEFT JOIN c ON r.user_id = c.user_id
),
rfc_scores AS (
    SELECT 
        user_id,
        recency,
        CASE 
            WHEN recency = '2017-12-03' THEN 100
            WHEN recency IN ('2017-12-02', '2017-12-01') THEN 80
            WHEN recency IN ('2017-11-30', '2017-11-29') THEN 60
            WHEN recency IN ('2017-11-28', '2017-11-27') THEN 40
            ELSE 20 
        END AS r_score,
        frequency,
        CASE 
            WHEN frequency > 15 THEN 100
            WHEN frequency BETWEEN 12 AND 14 THEN 90
            WHEN frequency BETWEEN 9 AND 11 THEN 70
            WHEN frequency BETWEEN 6 AND 8 THEN 50 
            WHEN frequency BETWEEN 3 AND 5 THEN 30
            ELSE 10 
        END AS f_score,
        cart_fav_count,
        CASE 
            WHEN cart_fav_count > 20 THEN 100
            WHEN cart_fav_count BETWEEN 16 AND 20 THEN 85
            WHEN cart_fav_count BETWEEN 11 AND 15 THEN 70
            WHEN cart_fav_count BETWEEN 6 AND 10 THEN 55 
            WHEN cart_fav_count BETWEEN 1 AND 5 THEN 40
            ELSE 20 
        END AS c_score
    FROM rfc_base
)

SELECT 
    t1.user_id,
    recency,
    r_score,
    avg_r,
    frequency,
    f_score,
    avg_f,
    cart_fav_count,
    c_score,
    avg_c,
    CASE    
        WHEN (f_score >= avg_f AND r_score >= avg_r AND c_score >= avg_c) THEN '价值用户'    
        WHEN (f_score >= avg_f AND r_score >= avg_r AND c_score < avg_c) THEN '潜力用户'    
        WHEN (f_score >= avg_f AND r_score < avg_r AND c_score >= avg_c) THEN '活跃用户'    
        WHEN (f_score >= avg_f AND r_score < avg_r AND c_score < avg_c) THEN '保持用户'    
        WHEN (f_score < avg_f AND r_score >= avg_r AND c_score >= avg_c) THEN '发展用户'    
        WHEN (f_score < avg_f AND r_score >= avg_r AND c_score < avg_c) THEN '新用户'    
        WHEN (f_score < avg_f AND r_score < avg_r AND c_score >= avg_c) THEN '兴趣用户'    
        ELSE '挽留用户'    
    END AS user_class     
FROM rfc_scores AS t1
LEFT JOIN 
(
    SELECT    
        user_id,    
        AVG(r_score) OVER() AS avg_r,    
        AVG(f_score) OVER() AS avg_f,
        AVG(c_score) OVER() AS avg_c    
    FROM    
        rfc_scores
) AS t2 ON t1.user_id = t2.user_id

RFC模型用户计数：

In [ ]:
create table df_rfc_count as 
select user_class,count(*) as user_class_num
from df_rfc 
group by user_class

## 4.3 商品分析  
### 4.3.1热卖商品  
统计商品和品类的销量，简单查询可知，一共有804826种商品和6867种品类，数量庞大，所以统计销量最多的top1000商品和top100品类。  
Top1000商品：

In [ ]:
create table product_buy_hot as 
select product_id,
count(if(action_type = 'buy',1,null)) as product_buy
from tbub 
group by product_id 
order by product_buy  desc 
limit 1000

Top100品类：

In [ ]:
create table category_buy_hot as 
select category_id,
count(if(action_type = 'buy',1,null)) as category_buy
from tbub 
group by category_id 
order by category_buy  desc 
limit 100

### 4.3.2品类决策时长  
以用户id和品类id分组，计算用户在特定品类第一次浏览到第一次购买的时间，将这段时间称为用户的决策时间。

In [ ]:
CREATE TABLE category_pv_buy_time AS
WITH 
-- 筛选有购买行为的用户-品类对
bought_categories AS (
    SELECT DISTINCT 
        user_id, 
        category_id
    FROM tbub
    WHERE action_type = 'buy'
),

-- 计算每个用户-品类的首次浏览时间
first_pv AS (
    SELECT 
        t.user_id,
        t.category_id,
        MIN(t.full_datetime) AS first_pv_time
    FROM tbub t
    JOIN bought_categories bc 
        ON t.user_id = bc.user_id AND t.category_id = bc.category_id
    WHERE 
        t.action_type = 'pv'
    GROUP BY 
        t.user_id, 
        t.category_id
),

-- 计算每个用户-品类的首次购买时间
first_buy AS (
    SELECT 
        t.user_id,
        t.category_id,
        MIN(t.full_datetime) AS first_buy_time
    FROM tbub t
    JOIN bought_categories bc 
        ON t.user_id = bc.user_id AND t.category_id = bc.category_id
    WHERE 
        t.action_type = 'buy'
    GROUP BY 
        t.user_id, 
        t.category_id
),

-- 计算每个用户-品类的转化时间
user_category_conversion AS (
    SELECT 
        p.user_id,
        p.category_id,
        p.first_pv_time,
        b.first_buy_time,
        TIMESTAMPDIFF(SECOND, p.first_pv_time, b.first_buy_time) AS conversion_seconds
    FROM first_pv p
    INNER JOIN first_buy b  -- 只保留有购买行为的记录
        ON p.user_id = b.user_id 
        AND p.category_id = b.category_id
        AND p.first_pv_time < b.first_buy_time  -- 确保浏览发生在购买前
),

-- 计算相同品类的平均转化时间
category_avg_conversion AS (
    SELECT 
        category_id,
        AVG(conversion_seconds) AS avg_conversion_seconds,
        AVG(conversion_seconds) / 3600 AS avg_conversion_hours  -- 转换为小时
    FROM user_category_conversion
    GROUP BY category_id
)

-- 最终结果：关联原始记录与品类平均时间
SELECT 
    ucc.user_id,
    ucc.category_id,
    ucc.first_pv_time,
    ucc.first_buy_time,
    ucc.conversion_seconds / 3600 AS conversion_hours,  -- 转为小时显示
    cat.avg_conversion_hours,
    (ucc.conversion_seconds / 3600) - cat.avg_conversion_hours AS hours_deviation  -- 与品类平均值的偏差
FROM user_category_conversion ucc
JOIN category_avg_conversion cat 
    ON ucc.category_id = cat.category_id
WHERE 
    ucc.first_pv_time > '2017-11-25'
ORDER BY 
    ucc.category_id,
    ucc.user_id;

### 4.3.3品类流量  
统计每个品类在每个小时的流量：

In [ ]:
create table category_hours_flow
select category_id,hours,
    sum(if(action_type='pv',1,0)) as pv,
    sum(if(action_type='cart',1,0)) as cart,
    sum(if(action_type='fav',1,0)) as fav,
    sum(if(action_type='buy',1,0)) as buy
from tbub 
group by category_id,hours
order by category_id,hours

统计每个品类在每天的流量：

In [ ]:
create table category_daily_flow
select category_id,dates,
sum(if(action_type='pv',1,0)) as pv,
sum(if(action_type='cart',1,0)) as cart,
sum(if(action_type='fav',1,0)) as fav,
sum(if(action_type='buy',1,0)) as buy
from tbub 
group by category_id,dates
order by category_id,dates

### 4.3.4 品类特征  
统计每个品类下用户行为的汇总：

In [ ]:
create table category_feature
select category_id,
        count(if(action_type='pv',1,null)) as PV,
        count(if(action_type='fav',1,null)) as FAV, 
        count(if(action_type='cart',1,null)) as CART, 
        count(if(action_type='buy',1,null)) as BUY
from tbub 
group by category_id

# 五、淘宝电商运营分析简报  
## 用户流量  
### 用户流量和留存受周末及 “双十二” 推流影响呈现波动  
* 图表日期从周六开始，可以看出工作日的浏览量（PV）比较稳定，而11-25，11-26和12-2，12-3这些周末时间的浏览量相对较高，用户闲暇时段购物需求释放。  
* 12-2和12-3的独立访客数（UV）和浏览量有显著的上升，可能是基于平台“双十二”活动大促蓄水、预热推流的策略。  
* 独立访客的平均浏览深度(PV/UV)在周末一般较高，但在“双十二”活动相对降低，因平台推流引入大量新客，稀释了平均浏览深度。  

![Image Name](https://cdn.kesci.com/upload/t03ssiomeb.png?imageView2/0/w/960/h/960)  

* 次日留存（Retention 1）在 12 - 1、12 - 2 显著攀升，三日留存（Retention 3）于 11 - 29、11 - 30 明显上扬 。二者趋势趋同，核心受双重因素驱动：一方面周末用户闲暇时间充裕，逛店后自然留存高；另一方面‘双十二’活动推流触达用户，通过优惠、内容吸引回访，共同助推留存率走高。  

![Image Name](https://cdn.kesci.com/upload/t03sutg2dw.png?imageView2/0/w/960/h/960)  



## 用户行为  
### 用户行为时序高度契合，10-16点及19-23点为在线高峰，20-22点尤甚  
* 针对用户收藏（FAV）、加购物车（CART)、购买（BUY）行为的时间序列分析，三种行为的时间序列波动高度契合，峰谷走势基本一致。  
* 每日的10:00-16:00和19:00-23:00是用户的在线高峰期，其中20:00-22:00是每日用户流量的最高峰期，或与用户下班后休闲购物时段匹配，运营可重点发力。  


![Image Name](https://cdn.kesci.com/upload/t0428u90k5.png)  



### 用户路径多样，直接浏览购买占比高，意向转成交流失大  
* 从用户路径可以看出（排除浏览-/-/-/），用户的购买路径复杂多样，其中收藏或加入购物车后不购买的情况占大多数。  
* 在有购买的路径中，占比从大到小依次是浏览-/-/-购买（16.7%）、浏览-/-加购-购买（8.6%）、浏览-收藏-/-购买（2.7%）、浏览-收藏-加购-购买（1.0%）。  
* 用户从意向（收藏 / 加购）到成交的转化链路有较大流失空间。  

![Image Name](https://cdn.kesci.com/upload/t0428az5q3.png)  


### 整体转化率2.0%-2.5%，需综合研判，周末及双十二因流量与运营存流失  
* 用户行为漏斗模型可以看出，整体转化率（UV-BUY）浮动在2.0%~2.5%之间，该转化率需结合行业、自身历史、业务阶段等综合分析是否正常。  
* 工作日的转化率较高，而周末和“双十二”活动流量大但转化低，或因流量质量（如大促凑单、周末闲逛用户多）、运营承接（大促链路长 / 优惠复杂）影响。  

![Image Name](https://cdn.kesci.com/upload/t0427s1fva.png)  


## 用户画像  
* 针对占比62%的挽留、新用户及潜力用户，分层施策激活，同步锁定价值用户并促兴趣用户转化  
* 由于缺乏销售金额数据，因此将RMF模型优化为RFC模型（最近购买时间R，购买频率F，收藏/加购物车频率C），对用户群体进行划分，划分为八个类别：  


![Image Name](https://cdn.kesci.com/upload/t042ihmcbh.png)  


* 8 类用户中，挽留用户（23%）、新用户（20%）、潜力用户（19%）合计占比 62%，构成用户主体；价值用户（12%）为核心利润来源，占比适中。  
* 挽留用户占比最高，反映大量用户长期沉寂，需优先低成本召回；活跃用户仅 5%，说明高忠诚度用户近期复购不足。  
* 新用户占比 20%，需通过首购后引导提升购买频率与加购收藏；潜力用户（19%）购买活跃但兴趣低，可通过关联推荐挖掘需求。  
* 价值用户（12%）需用专属权益锁定；兴趣用户（9%）加购收藏多但购买少，可针对性推送折扣促转化。  


![Image Name](https://cdn.kesci.com/upload/t042iv954m.png)  





## 品类分析  
* 平台商品超 8 万、品类近 7 千，品类销量差异大 ，聚焦热门品类  
* 平台商品总量达 804826 个、覆盖 6867 个品类，因个体数据（如销量 / 浏览量 ）分散，聚焦热门维度分析：按销量筛选 Top23 商品与品类，发现品类间流量差异远大于商品 。  
* 由于商品总体数量庞大、单个体量较小，而且无法得知具体商品和品类的名称，所以接下来主要分析各个热门品类为主。  


![Image Name](https://cdn.kesci.com/upload/t042aoojvg.png)  

![Image Name](https://cdn.kesci.com/upload/t042ayubfv.png)  


### 推测品类决策时长差异，负相关趋势下简化流程提转化  
* 不知具体品类名称，推测时间较长的品类，消费者决策更谨慎，如家具、电子产品；时间短的可能是冲动型消费品类或生活必需品类，像服饰、纸巾。  
* 观察平均浏览 - 购买时间和销量的整体趋势，整体呈负相关，要思考如何在短决策时间内提高转化率，如简化购买流程。  



![Image Name](https://cdn.kesci.com/upload/t04261c3j2.png)  




### Top3品类流量时段适配用户活跃，高浏览品类转化存漏，存在新老客差异  
* Top3 品类浏览量量 / 销量节奏与用户活跃时段（10-16、19-23 点，20-22 点峰值）高度重合，验证时段运营策略的普适性，也暴露品类间竞争的“同频内卷”。  
* 品类 4145813 浏览量断层领先，销量却未拉开差距，暴露出详情页转化力不足、价格竞争力弱等链路漏洞，需优先排查。  
* 品类 4145813和品类 2735466在11：00和21：00两个时间段的销量最高，但11：00的浏览量却只有21：00浏览量的1/2。推测可能21:00 浏览的是 “新客 / 犹豫客”（需大量信息辅助决策），11:00 下单的是 “老客 / 复购客”（依赖历史偏好，* * 无需重复浏览），两类用户群体无重叠。  


![Image Name](https://cdn.kesci.com/upload/t0425ftsfc.png)  



### 以浏览、销量均值划分四象限品类，归为畅销、高潜、利基、衰退类  
* 以浏览量为横轴，销量为纵轴，浏览量均值（42897）、销量均值（481）为象限分割线，可将品类划分四个类别。  
* 高浏览量、高销量（第一象限）的品类，流量大且转化强，用户认知度高、需求刚性，可能为快消标品（如纸巾、洗衣液 ）；  
* 高浏览量、低销量（第二象限）的品类，流量大但转化弱，用户决策成本高 / 需求伪高频，可能是高价耐用品（如大家电、首饰品 ）；  
* 低浏览量、高销量（第三象限）的品类，流量小但转化强，依赖精准需求 / 复购忠诚，比如专业刚需品（如办公耗材、工业零件 ）；  
* 低浏览量、低销量（第四象限）的品类，流量和需求双弱，产品同质化 / 市场需求小，可能是过气或冷门产品（如旧款功能机、手工藤编）。  
* 同一品类在不同阶段会跨象限，比如折叠屏手机的生命周期，第二象限：好奇高浏览、价高销量低 → 第一象限：成熟高浏览高销量 → 第四象限：功能落后低浏览低销量。  


![Image Name](https://cdn.kesci.com/upload/t0424tjzoc.png)  



# 补充：仪表盘  


![Image Name](https://cdn.kesci.com/upload/t0421rcyvk.png)  



## 总结  
在tableau可视化上没有展示具体实现方式，建议可以看看b站up主戴戴戴师兄的可视化教程。  
这个学习项目尽管已有不少前辈和同行做过实践，但我在借鉴他们成熟思路的基础上，也尝试加入了一些个人的思考与创新 —— 特别是在商品分析的维度设计和分析简报的呈现形式上，做了一些针对性的优化。  
不过深知还有很多可打磨的地方，今天把这些想法分享出来，一方面希望能给同样在做类似项目的朋友一些参考和启发，另一方面也特别期待能听到大家的建议！